[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_hand_detect_crop.ipynb)

In [ ]:
import cv2
import time
import os, gc
import mediapipe as mp
import gradio as gr
from threading import Thread
import math
from google.colab.patches import cv2_imshow
from PIL import Image
import numpy as np
example_flag = False

# https://huggingface.co/spaces/datasciencedojo/Finger-Counting-Right-Hand/blob/main/app.py modified 
class handDetector():
    def __init__(self, mode=True, modelComplexity=1, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.modelComplex = modelComplexity
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,self.modelComplex,self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True, flipType=True):
        """
        Finds hands in a BGR image.
        :param img: Image to find the hands in.
        :param draw: Flag to draw the output on the image.
        :return: Image with or without drawings
        """
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #cv2.imshow('test',imgRGB)
        self.results = self.hands.process(imgRGB)
        allHands = []
        h, w, c = img.shape
        if self.results.multi_hand_landmarks:
            for handType, handLms in zip(self.results.multi_handedness, self.results.multi_hand_landmarks):
                myHand = {}
                ## lmList
                mylmList = []
                xList = []
                yList = []
                for id, lm in enumerate(handLms.landmark):
                    px, py, pz = int(lm.x * w), int(lm.y * h), int(lm.z * w)
                    mylmList.append([px, py, pz])
                    xList.append(px)
                    yList.append(py)

                ## bbox
                xmin, xmax = min(xList), max(xList)
                ymin, ymax = min(yList), max(yList)
                boxW, boxH = xmax - xmin, ymax - ymin
                bbox = xmin, ymin, boxW, boxH
                cx, cy = bbox[0] + (bbox[2] // 2), \
                         bbox[1] + (bbox[3] // 2)

                myHand["lmList"] = mylmList
                myHand["bbox"] = bbox
                myHand["center"] = (cx, cy)

                if flipType:
                    if handType.classification[0].label == "Right":
                        myHand["type"] = "Left"
                    else:
                        myHand["type"] = "Right"
                else:
                    myHand["type"] = handType.classification[0].label
                allHands.append(myHand)

                if draw:
                    # self.mpDraw.draw_landmarks(img, handLms,self.mpHands.HAND_CONNECTIONS)
                    # cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20), (bbox[0] + bbox[2] + 20, bbox[1] + bbox[3] + 20), (255, 0, 255), 10)

                    center_x = bbox[0] + bbox[2] // 2
                    center_y = bbox[1] + bbox[3] // 2
                    center = (center_x, center_y)
                    # cv2.circle(img, center, radius=5, color=(255, 0, 255), thickness=10)
                    length = math.sqrt(bbox[2]**2 + bbox[3]**2)
                    x = int(center_x - int(length//2))
                    y = int(center_y - int(length//2))

                    if x < 0:
                        x = 0
                    if y < 0:
                        y = 0
                    if x + int(length) > img.shape[1]:
                        x = img.shape[1] - int(length)
                    if y + int(length) > img.shape[0]:
                        y = img.shape[0] - int(length)
                    # cv2.rectangle(img, (x, y), (x+int(length), y+int(length)), (0, 0, 255), 10)

                    width, height = int(length), int(length)
                    x = center_x - (width // 2)
                    y = center_y - (height // 2)
                    
                    if x < 0:
                        x = 0
                    if y < 0:
                        y = 0
                    if x + width > img.shape[1]:
                        x = img.shape[1] - width
                    if y + height > img.shape[0]:
                        y = img.shape[0] - height
                    img = img[y:y+height, x:x+width]
                    img = cv2.resize(img, (512, 512))

        if draw:
            return allHands, img
        else:
            return allHands

def multi(image_path):
    img = cv2.imread(image_path) 
    image_size = img.shape[:2]
    if image_size == (512, 512):
        print("512x512")
        return img
    else:
        detector = handDetector(detectionCon=0.75)
        allhands, img = detector.findHands(img, 1)
        del detector
        gc.collect()
        cv2.imwrite(image_path, img[:,:,::-1]) 
        return img

def get_image_paths(folder_path):
    image_paths = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith(('.jpg', '.png', '.jpeg')):
                file_path = os.path.join(dirpath, filename)
                image_paths.append(file_path)
    return image_paths

root_folder_path = '/content/images'
all_image_paths = get_image_paths(root_folder_path)

for image_path in all_image_paths:
    multi(image_path)
    print(image_path)